In [1]:
import xarray as xr
import numpy as np

In [2]:
varlist = ['hurs','psl','rls','rss', 'sfcWind','tasmax','tasmin','tas']
pathformat = '/Users/nelereyniers/data/toydata/toydata_3x3/{var}_rcp85_land-rcm_uk_12km_01_day_19801201-20801130.nc'
indsdict = dict([[v, xr.open_dataset(pathformat.format(var=v))] for v in varlist])
outpath = '/Users/nelereyniers/data/toydata/toydata_3x3/evpot-fao_test_rcp85_land-rcm_uk_12km_01_day_19801201-20801130.nc'
etds = xr.open_dataset(outpath)

In [11]:
print('eto:  ', etds['evpot-fao_test'].isel(projection_x_coordinate=2, 
                                               projection_y_coordinate=2,
                                               time=2).values)
test = {}
for v in varlist:
    test[v]=indsdict[v][v].isel(projection_x_coordinate=2, 
                                projection_y_coordinate=2,
                                time=2).values
print(test)

eto:   [0.08]
{'hurs': array([82.85444], dtype=float32), 'psl': array([1015.0265], dtype=float32), 'rls': array([-83.24196], dtype=float32), 'rss': array([40.505985], dtype=float32), 'sfcWind': array([4.529573], dtype=float32), 'tasmax': array([2.0761194], dtype=float32), 'tasmin': array([-1.6807618], dtype=float32), 'tas': array([-0.2098756], dtype=float32)}


In [12]:
indsdict['hurs']['hurs'][0, 0].time

<xarray.DataArray 'time' ()>
array(cftime.Datetime360Day(1980, 12, 1, 12, 0, 0, 0), dtype=object)
Coordinates:
    ensemble_member     int32 1
    ensemble_member_id  |S27 ...
    time                object 1980-12-01 12:00:00
    month_number        int32 ...
    year                int32 ...
    yyyymmdd            |S64 ...
Attributes:
    axis:           T
    bounds:         time_bnds
    standard_name:  time

In [13]:
etds['evpot-fao_test'][0,0,].values

array([[0.46, 0.53, 0.56],
       [0.45, 0.49, 0.46],
       [0.43, 0.43, 0.45]])

# Calculating ETo step by step avaluesheck

In [14]:
def es_from_T(T):
    return 0.6108*np.exp(17.27*T/(T+237.3))

def u2_from_uz(uz, z=10):
    return uz*4.87/np.log(67.8*z-5.42)

In [15]:
es = (es_from_T(test['tasmin']) + es_from_T(test['tasmax']))/2
delta = 4098*(0.6108*np.exp(17.27*test['tas']/(test['tas']+237.3)))/((test['tas']+237.3)**2)
ea = es*test['hurs']/100
G=0
u2 = u2_from_uz(test['sfcWind'])
gamma = 0.665*10**-3*(test['psl']/10)
Rn = (test['rss'] + test['rls'])* 0.0864

In [16]:
eto = (0.408*delta*(Rn - G) + gamma*900/(test['tas']+273)*u2*(es-ea))/(delta + gamma*(1+0.34*u2)) 

In [17]:
eto

array([0.07800725], dtype=float32)

In [81]:
es  # match

array([0.9018941], dtype=float32)

In [36]:
delta  # match

array([0.06319913], dtype=float32)

In [84]:
test['sfcWind']

array([4.9960027], dtype=float32)

In [83]:
u2  # not a match! but this one is correct.

array([3.7367654], dtype=float32)

In [40]:
test['sfcWind']

array([4.9960027], dtype=float32)

In [58]:
gamma

array([0.06614523], dtype=float32)

In [59]:
indsdict['sfcWind']['sfcWind']

<xarray.DataArray 'sfcWind' (ensemble_member: 1, time: 36000, projection_y_coordinate: 3, projection_x_coordinate: 3)>
[324000 values with dtype=float32]
Coordinates:
  * ensemble_member          (ensemble_member) int32 1
  * projection_y_coordinate  (projection_y_coordinate) float64 1.98e+05 ... 2.22e+05
  * projection_x_coordinate  (projection_x_coordinate) float64 5.58e+05 ... 5.82e+05
    ensemble_member_id       (ensemble_member) |S27 ...
    grid_latitude            (projection_y_coordinate, projection_x_coordinate) float64 ...
    grid_longitude           (projection_y_coordinate, projection_x_coordinate) float64 ...
  * time                     (time) object 1980-12-01 12:00:00 ... 2080-11-30 12:00:00
    month_number             (time) int32 ...
    year                     (time) int32 ...
    yyyymmdd                 (time) |S64 ...
Attributes:
    standard_name:  wind_speed
    long_name:      Wind speed at 10m
    units:          m s-1
    description:    Wind speed
    label_units:    m s-1
    plot_label:     Wind speed at 10m (m s-1)
    cell_methods:   time: mean
    grid_mapping:   transverse_mercator

In [48]:
indsdict['psl']['psl']

<xarray.DataArray 'psl' (ensemble_member: 1, time: 36000, projection_y_coordinate: 3, projection_x_coordinate: 3)>
[324000 values with dtype=float32]
Coordinates:
  * ensemble_member          (ensemble_member) int32 1
  * projection_y_coordinate  (projection_y_coordinate) float64 1.98e+05 ... 2.22e+05
  * projection_x_coordinate  (projection_x_coordinate) float64 5.58e+05 ... 5.82e+05
    ensemble_member_id       (ensemble_member) |S27 b'HadREM3-GA705-r001i1p00000 '
    grid_latitude            (projection_y_coordinate, projection_x_coordinate) float64 2.214 ... 2.364
    grid_longitude           (projection_y_coordinate, projection_x_coordinate) float64 -10.89 ... -10.62
  * time                     (time) object 1980-12-01 12:00:00 ... 2080-11-30 12:00:00
    month_number             (time) int32 12 12 12 12 12 12 ... 11 11 11 11 11
    year                     (time) int32 1980 1980 1980 1980 ... 2080 2080 2080
    yyyymmdd                 (time) |S64 b'19801201                                                        ' ... b'20801130                                                        '
Attributes:
    standard_name:  air_pressure_at_sea_level
    long_name:      Pressure at mean sea level
    units:          hPa
    description:    Sea level pressure
    label_units:    hPa
    plot_label:     Sea level pressure (hPa)
    cell_methods:   time: mean
    grid_mapping:   transverse_mercator